In [1]:
pip install spacy

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xgboost

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install numpy

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install scipy

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install sklearn

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install squarify

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install eli5

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install category_encoders

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import numpy as np
import matplotlib as plt
from scipy import stats

In [11]:
from xgboost import XGBClassifier as XGBoost

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
from sklearn.pipeline import make_pipeline

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
import eli5
from eli5.sklearn import PermutationImportance

In [17]:
import time

In [18]:
from collections import Counter

In [19]:
import squarify
import matplotlib.pyplot as plt

In [20]:
import category_encoders as ce

In [21]:
from sklearn.metrics import mean_absolute_error as mae

In [22]:
import spacy

from spacy.tokenizer import Tokenizer

In [23]:
%%time
nlp = spacy.load("en_core_web_lg")

CPU times: user 9.28 s, sys: 1.25 s, total: 10.5 s
Wall time: 10.5 s


# DATA WRANGLING PORTION

In [24]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [25]:
# columns= ['id','neighbourhood_group_cleansed','property_type','room_type','price',
#           'minimum_nights','bathrooms','bedrooms','beds','bed_type','guests_included',
#           'accommodates','amenities','security_deposit','cleaning_fee','extra_people',
#           'instant_bookable','is_business_travel_ready','cancellation_policy']

# # NUMERIC ONLY
# columns= ['price',
#           'minimum_nights','bathrooms','bedrooms','beds','guests_included',
#           'accommodates','security_deposit','cleaning_fee']

# NUMERIC ONLY
columns= ['price','room_type','neighbourhood_group_cleansed',
          'minimum_nights','bathrooms','bedrooms','beds','bed_type',
          'instant_bookable','accommodates','security_deposit','cleaning_fee']

In [26]:
df = pd.read_csv(r'./listings_summary.csv',
                 usecols=columns)

In [27]:
df.head()

,neighbourhood_group_cleansed,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,security_deposit,cleaning_fee,minimum_nights,instant_bookable
0,Mitte,Entire home/apt,3,1.0,1.0,2.0,Real Bed,$60.00,$200.00,$30.00,4,f
1,Pankow,Private room,2,1.0,1.0,1.0,Real Bed,$17.00,$0.00,$0.00,2,f
2,Pankow,Entire home/apt,4,1.0,1.0,2.0,Real Bed,$90.00,$200.00,$50.00,62,t
3,Tempelhof - Schöneberg,Private room,2,1.0,1.0,1.0,Pull-out Sofa,$26.00,$250.00,$30.00,5,f
4,Pankow,Private room,2,1.0,1.0,2.0,Real Bed,$42.00,$0.00,$0.00,2,f


In [28]:
df.describe()

,accommodates,bathrooms,bedrooms,beds,minimum_nights
count,22552.000000,22520.000000,22534.000000,22512.000000,22552.000000
mean,2.637460,1.092695,1.161134,1.620558,7.157059
std,1.510548,0.329894,0.646156,1.174840,40.665073
min,1.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,1.000000,1.000000,1.000000,2.000000
50%,2.000000,1.000000,1.000000,1.000000,2.000000
75%,3.000000,1.000000,1.000000,2.000000,4.000000
max,16.000000,8.500000,12.000000,22.000000,5000.000000


In [29]:
len(df)

22552

In [30]:
df['price'] = df['price'].str.replace('$','')
df['price'] = df['price'].str.replace(',','')
df['price'] = df['price'].str.replace('.00','')

In [31]:
df['security_deposit'] = df['security_deposit'].str.replace('$','')
df['security_deposit'] = df['security_deposit'].str.replace(',','')
df['security_deposit'] = df['security_deposit'].str.replace('.00','')

In [32]:
df['cleaning_fee'] = df['cleaning_fee'].str.replace('$','')
df['cleaning_fee'] = df['cleaning_fee'].str.replace(',','')
df['cleaning_fee'] = df['cleaning_fee'].str.replace('.00','')

In [33]:
df['price'] = pd.to_numeric(df['price'])
df['security_deposit'] = pd.to_numeric(df['security_deposit'])
df['cleaning_fee'] = pd.to_numeric(df['cleaning_fee'])
# df['extra_people'] = pd.to_numeric(df['extra_people'])

In [34]:
df = df[df['minimum_nights'] <= 7]

In [35]:
df = df[df['price'] <= 300]

In [36]:
len(df)

19499

In [37]:
df.dtypes

neighbourhood_group_cleansed     object
room_type                        object
accommodates                      int64
bathrooms                       float64
bedrooms                        float64
beds                            float64
bed_type                         object
price                           float64
security_deposit                float64
cleaning_fee                    float64
minimum_nights                    int64
instant_bookable                 object
dtype: object

In [38]:
df.isna().sum()

neighbourhood_group_cleansed        0
room_type                           0
accommodates                        0
bathrooms                          27
bedrooms                           15
beds                               34
bed_type                            0
price                               0
security_deposit                12582
cleaning_fee                     6250
minimum_nights                      0
instant_bookable                    0
dtype: int64

In [39]:
len(df)

19499

In [40]:
# Dropping rows where price is NaN
df = df.dropna(subset=['price','bathrooms','bedrooms','beds'])

In [41]:
len(df)

19432

In [42]:
df['security_deposit'] = df['security_deposit'].replace(to_replace=np.NaN, value=0)
df['cleaning_fee'] = df['cleaning_fee'].replace(to_replace=np.NaN, value=0)
# df['extra_people'] = df['extra_people'].replace(to_replace=np.NaN, value=0)

In [43]:
neighbourhood_remap = {'Friedrichshain-Kreuzberg': 1, 'Mitte': 2, 'Neukölln': 3,'Pankow': 4, 'Tempelhof - Schöneberg': 0,
                       'Charlottenburg-Wilm.':0,'Lichtenberg':0,'Treptow - Köpenick':0,'Steglitz - Zehlendorf':0,
                       'Reinickendorf':0,'Marzahn - Hellersdorf':0,'Spandau':0}

In [44]:
df.replace({'neighbourhood_group_cleansed': neighbourhood_remap}, inplace = True)

In [45]:
df['neighbourhood_group_cleansed'].astype(int);

In [46]:
df = df[df['neighbourhood_group_cleansed'] !=0]

In [47]:
room_type_remap = {'Private room':1,'Entire home/apt': 2,'Shared room': 3}

In [48]:
df.replace({'room_type': room_type_remap}, inplace=True)

In [49]:
df['room_type'].astype(int);

In [50]:
bed_type_remap = {'Real Bed': 1,'Pull-out Sofa': 2,'Futon': 3,'Couch':4,'Airbed':5}

In [51]:
df.replace({'bed_type': bed_type_remap}, inplace=True)

In [52]:
df['bed_type'].astype(int);

In [53]:
df['instant_bookable']= df['instant_bookable'].replace({'f': 0, 't': 1})

In [54]:
target = 'price'
security_drop = 'security_deposit'
cleaning_drop = 'cleaning_fee'
X = df.drop(columns=target)
# X2 = X3.drop(columns=security_drop)
# X = X2.drop(columns=cleaning_drop)
y = df[target]

In [55]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [56]:
X_train.head()

,neighbourhood_group_cleansed,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,security_deposit,cleaning_fee,minimum_nights,instant_bookable
13287,2,2,4,1.0,1.0,2.0,1,0.0,20.0,3,1
20737,4,1,2,1.0,1.0,2.0,1,0.0,0.0,1,0
11517,2,2,4,1.0,1.0,2.0,1,0.0,50.0,2,1
3537,3,2,5,1.0,3.0,5.0,1,0.0,60.0,4,0
2809,2,1,5,1.0,1.0,3.0,1,0.0,35.0,2,0


In [57]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((9966, 11), (9966,), (4910, 11), (4910,))

In [58]:
X_train.columns

Index(['neighbourhood_group_cleansed', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'security_deposit', 'cleaning_fee', 'minimum_nights', 'instant_bookable'], dtype='object')

In [59]:
df.head()

,neighbourhood_group_cleansed,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,security_deposit,cleaning_fee,minimum_nights,instant_bookable
0,2,2,3,1.0,1.0,2.0,1,60.0,0.0,30.0,4,0
1,4,1,2,1.0,1.0,1.0,1,17.0,0.0,0.0,2,0
4,4,1,2,1.0,1.0,2.0,1,42.0,0.0,0.0,2,0
5,4,2,7,2.5,4.0,7.0,1,180.0,0.0,80.0,6,0
9,4,1,2,1.0,1.0,1.0,1,45.0,0.0,18.0,3,0


In [60]:
# encoder = ce.OrdinalEncoder(cols=['neighbourhood_group_cleansed','room_type'])

In [61]:
# pipeline = make_pipeline(
# #     ce.OrdinalEncoder(cols=['neighbourhood_group_cleansed','room_type']),
#     ce.OneHotEncoder(use_cat_names=True),
#     RandomForestClassifier(max_depth = 20, min_samples_split = 10, n_estimators = 100, random_state = 42, n_jobs = -1)
# )

In [62]:
# pipeline.fit(X_train, y_train)

In [63]:
# print(pipeline.score(X_test,y_test))

In [64]:
# X_train_processed = processor.fit_transform(X_train)

In [65]:
# X_test_processed = processor.transform(X_test)

In [66]:
# eval_set = [(X_train_processed, y_train),
#             (X_test_processed, y_test)]

In [67]:
# model =  XGBoost(n_estimators = 10000, n_jobs=-1)

In [68]:
# model.fit(X_train_processed, y_train, eval_set=eval_set, eval_metric='mae',
#          early_stopping_rounds=10)

In [69]:
# encoder.fit(X,y)

In [70]:
# X_cleaned = encoder.transform(X)

In [71]:
%%time
pipeline = make_pipeline(
    StandardScaler(),
    XGBoost(n_estimators=100, random_state=42, n_jobs=-1)
)

pipeline.fit(X_train, y_train)
print('Validation Accuracy', pipeline.score(X_test, y_test))

Validation Accuracy 0.09429735234215886
CPU times: user 2min 46s, sys: 668 ms, total: 2min 47s
Wall time: 24.8 s


In [72]:
%%time
model = XGBoost(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

CPU times: user 2min 26s, sys: 740 ms, total: 2min 27s
Wall time: 21.4 s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=-1,
              nthread=None, objective='multi:softprob', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [73]:
%%time
permuter = PermutationImportance(
    model, scoring='accuracy', n_iter=2, random_state=42
)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 434 µs


In [74]:
%%time
permuter.fit(X_test, y_test)
feature_names = X_test.columns.tolist()
eli5.show_weights(
    permuter,
    top=None,
    feature_names = feature_names
)

CPU times: user 7min 34s, sys: 88 ms, total: 7min 34s
Wall time: 1min 8s


Weight,Feature
0.0219 ± 0.0043,room_type
0.0129 ± 0.0006,cleaning_fee
0.0108 ± 0.0053,accommodates
0.0056 ± 0.0002,minimum_nights
0.0042 ± 0.0026,neighbourhood_group_cleansed
0.0037 ± 0.0016,instant_bookable
0.0018 ± 0.0020,bedrooms
0.0015 ± 0.0022,security_deposit
0.0003 ± 0.0043,beds
-0.0001 ± 0.0010,bed_type


In [75]:
%%time
1+1

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.48 µs


2

In [76]:
y_pred=pipeline.predict(X_test)

In [77]:
y_pred

array([ 35.,  70., 110., ...,  35., 190.,  30.])

In [78]:
s = pd.Series(y_pred)

In [79]:
s

0        35.0
1        70.0
2       110.0
3        25.0
4        30.0
        ...  
4905     50.0
4906     50.0
4907     35.0
4908    190.0
4909     30.0
Length: 4910, dtype: float64

In [80]:
y_test

19375     75.0
5412      55.0
9091     140.0
14950     42.0
13416     65.0
         ...  
16664     45.0
15922     44.0
11815     60.0
132      170.0
10577     38.0
Name: price, Length: 4910, dtype: float64

In [81]:
print('Mean Absolute Error', mae(y_test, y_pred))

Mean Absolute Error 18.325050916496945


In [82]:
y.shape, y_pred.shape

((14876,), (4910,))

# NLP PORTION

In [83]:
# %%time
# # Tokenizer
# tokenizer = Tokenizer(nlp.vocab)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 33.4 µs


In [84]:
# df.head()

,neighbourhood_group_cleansed,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,security_deposit,cleaning_fee,minimum_nights,instant_bookable
0,2,2,3,1.0,1.0,2.0,1,60.0,0.0,30.0,4,0
1,4,1,2,1.0,1.0,1.0,1,17.0,0.0,0.0,2,0
4,4,1,2,1.0,1.0,2.0,1,42.0,0.0,0.0,2,0
5,4,2,7,2.5,4.0,7.0,1,180.0,0.0,80.0,6,0
9,4,1,2,1.0,1.0,1.0,1,45.0,0.0,18.0,3,0


In [85]:
# df['amenities'].iloc[0]

KeyError: 'amenities'

In [ ]:
# %%time
# # Tokenizer
# tokenizer = Tokenizer(nlp.vocab)

In [ ]:
# STOPS = ['}','{','"',"'"]

In [ ]:
# PARSES THROUGH ONE INDEX LOCATION OF AMENITIES
# cool1 = []
# cool = []

# for a in df['amenities'].iloc[0]:
#     if a not in STOPS:
#         cool.extend(a)
# cool1 = ''.join(cool)

In [ ]:
# cool1 = []
# for doc in tokenizer.pipe(df['amenities'], batch_size=1000):
#     cool = []
#     for a in df['amenities']:
#         if a not in STOPS:
#             cool.extend(a)
#     cool1.append(cool)

In [ ]:
# df['cool'] = cool1

In [ ]:
# df['cool'].head()

In [ ]:
# cool

In [ ]:
# cool1

In [ ]:
# STOP_WORDS = nlp.Defaults.stop_words.union(['\{','{','"'])

In [ ]:
# %%time
# # Wrap it all in a function
# def get_lemmas(text):

#     lemmas = []
    
#     doc = nlp(text)
    
#     # Something goes here :P
#     for token in doc: 
#         if token.text not in STOP_WORDS:
#             if ((token.is_stop == False) and (token.is_punct == False)) and ((token not in STOP_WORDS) and (token.pos_!= 'PRON')):
#                 lemmas.append(token.lemma_)
    
#     return lemmas

In [ ]:
# tokens = []

In [ ]:
# import re

In [ ]:
# %%time
# for doc in tokenizer.pipe(cool1, batch_size=1000):
    
#     doc_tokens = []
    
#     for token in doc: 
# #             doc_tokens_text = re.sub(r'[^a-zA-Z ^0-9]', '', token.text)
#         doc_tokens_text1 = token.text.replace('{','')
#         doc_tokens_text2 = doc_tokens_text1.replace('}','')
#         doct_tokens_text3 = doc_tokens_text2.replace('"','')
#         doc_tokens.append(doc_tokens_text1)
#     tokens.append(doc_tokens)
    
# df['tokens'] = tokens

In [ ]:
# doc_tokens

In [ ]:
# tokens[2]

In [ ]:
# tokens.unique()

In [ ]:
# df.head()

In [ ]:
# df.isna().sum()

In [ ]:
# df['amenities'].head()

In [ ]:
# df['tokens'].head()

In [ ]:
# %%time
# df['lemmas'] = df['amenities'].apply(get_lemmas)

In [ ]:
# df['lemmas'].head()

In [ ]:
# def count(docs):

#         word_counts = Counter()
#         appears_in = Counter()
        
#         total_docs = len(docs)

#         for doc in docs:
#             word_counts.update(doc)
#             appears_in.update(set(doc))

#         temp = zip(word_counts.keys(), word_counts.values())
        
#         wc = pd.DataFrame(temp, columns = ['word', 'count'])

#         wc['rank'] = wc['count'].rank(method='first', ascending=False)
#         total = wc['count'].sum()

#         wc['pct_total'] = wc['count'].apply(lambda x: x / total)
        
#         wc = wc.sort_values(by='rank')
#         wc['cul_pct_total'] = wc['pct_total'].cumsum()

#         t2 = zip(appears_in.keys(), appears_in.values())
#         ac = pd.DataFrame(t2, columns=['word', 'appears_in'])
#         wc = ac.merge(wc, on='word')

#         wc['appears_in_pct'] = wc['appears_in'].apply(lambda x: x / total_docs)
        
#         return wc.sort_values(by='rank')

In [ ]:
# def tokenize(text):
#     tokens = []

#     for doc in tokenizer.pipe(text):

#         doc_tokens = []

#         for token in doc: 
#             if token.text not in STOP_WORDS:
#                 doc_tokens_text = re.sub(r'[^a-zA-Z ^0-9]', '', token.text)
#                 doc_tokens_text = doc_tokens_text.split()
# #                 doc_tokens.append(doc_tokens_text.lower())


#         tokens.append(doc_tokens)
#     return tokens

In [ ]:
# df['tokens'] = tokenize(df['amenities'])

In [ ]:
# df['tokens']

In [ ]:
# wc = count(df['tokens'])
# wc = count(df['tokens'])

In [ ]:
# wc_top20 = wc[wc['rank'] <= 20]

In [ ]:
# wc_top20['word'].sort_values(ascending=True)

In [ ]:
# %%time
# squarify.plot(sizes=wc_top20['pct_total'], label=wc_top20['word'], alpha=.8)
# plt.axis('off')
# plt.show()

### Counting Amenities, Encoding Neighborhood_group_cleansed, Property_type

In [ ]:
# df['neighbourhood_group_cleansed'].value_counts()

In [ ]:
# s = df['neighbourhood_group_cleansed'].value_counts()
# df = df[df.isin(s.index[s>=1000]).values]

In [ ]:
# df['room_type'].value_counts()

In [ ]:
# s = df['room_type'].value_counts()
# df = df[df.isin(s.index[s>=60])]

In [ ]:
# len(df)

# KERAS NEURAL NETWORK PORTION

In [ ]:
# pip install tensorflow

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense

# np.random.seed(812)

In [ ]:
# model = Sequential()
# model.add(Dense(1, input_dim=9, activation='relu', name="Dense1"))
# model.add(Dense(3, activation='relu', name = 'Dense2'))
# model.add(Dense(1, activation='relu'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(X_train,y_train, epochs=5)

In [ ]:
# model.summary()